In [1]:
from feat_competition_sales import *

from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import xgboost as xgb

/home/stas/anaconda3/envs/comp1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning:

Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.



In [2]:
# Tell plotly to plot in a notebook
offline.init_notebook_mode(connected=True)

### Load and plot data

In [3]:
# Load data
store = pd.read_csv('data/store.csv')
train = pd.read_csv('data/train.csv', dtype = {'StateHoliday': np.str})

# Merge
train = pd.merge(store, train, on='Store')
train = date_convert(train)

# Split
end_val = np.floor(0.8 * train.shape[0]).astype(int)
end_test = np.floor(0.9 * train.shape[0]).astype(int)

Train = train.loc[:end_val]
Val = train.loc[end_val:end_test]
Test = train.loc[end_test:]

In [5]:
#plot_hists(train, name='train')

In [ ]:
#plot_hists(store, name='store')

### Feature engineering 

In [4]:
# Clean data, construct new features for xgboost
Train = data_transformation(Train)
Val = data_transformation(Val)
Test = data_transformation(Test)

/home/stas/anaconda3/envs/comp1/lib/python3.6/site-packages/pandas/core/series.py:853: RuntimeWarning:

invalid value encountered in log



In [6]:
# OHE
cols = Train.select_dtypes(include='object').columns.tolist()

for col in cols:
    Train[col] = Train[col].astype(str)
    Val[col] = Val[col].astype(str)
    Test[col] = Test[col].astype(str)

ohe = OneHotEncoder(handle_unknown='ignore')
ohe_train = pd.DataFrame()
ohe_val = pd.DataFrame()
ohe_test = pd.DataFrame()


for col in cols:
    ohe.fit(np.array(Train[col]).reshape(-1,1))
    ohe_train_tmp = pd.DataFrame(columns=ohe.categories_, \
                                 data=ohe.transform(np.array(Train[col]).reshape(-1,1)).toarray())
    ohe_val_tmp = pd.DataFrame(columns=ohe.categories_, \
                                 data=ohe.transform(np.array(Val[col]).reshape(-1,1)).toarray()) 
    ohe_test_tmp = pd.DataFrame(columns=ohe.categories_, \
                                 data=ohe.transform(np.array(Test[col]).reshape(-1,1)).toarray()) 
    ohe_train = pd.concat([ohe_train, ohe_train_tmp], axis=1)
    ohe_val = pd.concat([ohe_val, ohe_val_tmp], axis=1)
    ohe_test = pd.concat([ohe_test, ohe_test_tmp], axis=1)

In [7]:
# Drop columns
Train.drop(axis=1, labels=cols, inplace=True)
Val.drop(axis=1, labels=cols, inplace=True)
Test.drop(axis=1, labels=cols, inplace=True)

In [10]:
# Concat
Train = pd.concat([Train.reset_index(), ohe_train], axis=1)
Val = pd.concat([Val.reset_index(), ohe_val], axis=1)
Test = pd.concat([Test.reset_index(), ohe_test], axis=1)

In [13]:
# Write data to csv to reuse it fast
#Train.to_csv('data/stasTrain.csv', index=False)
#Test.to_csv('data/stasTest.csv', index=False)
#Val.to_csv('data/stasVal.csv', index=False)

In [34]:
Train = pd.read_csv('data/stasTrain.csv', index_col=False).drop(axis=1, labels='index')
Val = pd.read_csv('data/stasVal.csv', index_col=False).drop(axis=1, labels='index')
Test = pd.read_csv('data/stasTest.csv', index_col=False).drop(axis=1, labels='index')

In [16]:
# Create X_train, y_train and so on
X_train = Train.drop(axis=1, labels=['Sales, Index'])
X_train = X_train
y_train = Train['Sales']
X_val = Val.drop(axis=1, labels=['Sales, Index'])
y_val = Val['Sales']
X_test = Test.drop(axis=1, labels=['Sales, Index,'])
y_test = Test['Sales']

### Training

In [71]:
# XGBoost
params = {
        'max_depth': [3, 4, 5, 6, 7],
        'learning_rate': [0.001, 0.05, 0.1],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
        'colsample_bylevel': [0.3, 0.5, 0.6, 0.7, 0.8],
        'reg_lambda' : [0.1, 0.5, 1.0, 2, 3, 4, 5],
        'n_estimators': [1000, 4000]
}

ind = np.random.randint(0, X_train.shape[0], size=50000)
xgb_params = parameter_search(X_train.iloc[ind], y_train.iloc[ind], method='xgboost', params=params, steps=50)


  0%|          | 0/50 [00:00<?, ?it/s]

(0.05, 2489, 3, 0.8, 0.7, 0.3, 5.0)



  2%|▏         | 1/50 [00:37<30:34, 37.44s/it]

(0.1, 3520, 6, 0.8, 0.6, 0.8, 0.5)



  4%|▍         | 2/50 [01:12<29:25, 36.77s/it]

...


 96%|█████████▌| 48/50 [53:31<02:10, 65.37s/it]

(0.001, 3407, 4, 0.8, 0.3, 0.5, 2.0)



 98%|█████████▊| 49/50 [55:18<01:17, 77.99s/it]

(0.05, 2497, 3, 0.8, 0.8, 0.3, 4.0)



100%|██████████| 50/50 [55:56<00:00, 67.14s/it]


In [154]:
# RF
params = {
        'max_depth': [3, 4, 5, 6, 7],
        'n_estimators': [5000, 15000],
        'max_features': [0.8, 0.9, 1]
}

ind = np.random.randint(0, X_train.shape[0], size=50000)
rf_params = parameter_search(X_train.iloc[ind], y_train.iloc[ind], method='rf', params=params, steps=25)













  0%|          | 0/25 [00:00<?, ?it/s]

(11276, 3, 0.8)














  4%|▍         | 1/25 [09:06<3:38:42, 546.77s/it]

(12516, 7, 0.9)














  8%|▊         | 2/25 [32:25<5:07:34, 802.38s/it]

...













 76%|███████▌  | 19/25 [4:26:57<1:41:55, 1019.23s/it]

(11878, 5, 0.9)














 80%|████████  | 20/25 [4:43:49<1:24:44, 1016.87s/it]

(10247, 6, 1.0)


KeyboardInterrupt: 

In [21]:
# Perform PCA
pca = PCA(n_components=4)
pca.fit(X_train)def predict_stacked(X, models, coefs):
    pca = pickle.load(open("pca.pickle.dat", "rb"))
    X_pca = pca.transform(X)
    valid_models = {'xgb', 'rf', 'el_net'}
    if models.keys() not in valid_models:
        raise ValueError("Model names must be one of %r." % valid_models)
        
    if not type(models) == dict & type(coefs) == dict:
        raise ValueError('models and coefs are dictionaries, please support a dictionary')
    else:    
        xgb_mod = models['xgb']
        rf_mod = models['rf']
        el_net_mod = models['el_net']
        xgb_coef = coefs['xgb']
        rf_coef = coefs['rf']
        el_net_coef = coefs['el_net']
    
    return np.sum(xgb_coef * xgb_mod.predict(X) + rf_coed * rf_mod.predict(X) + el_net_coef * el_net.predict(X_pca))     
pickle.dump(pca, open("pca.pickle.dat", "wb"))

In [34]:
# Elastic Net
params = {
        'ratio': np.linspace(0.1, 1, 20),
        'alpha': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 5, 10, 20, 50, 100],
        'max_features': [0.8, 0.9, 1]
}

el_net_params = parameter_search(X_train_pca, y_train, method='elastic', params=params)

0.1 1e-05
0.1 5e-05
0.1 0.0001
0.1 0.0005
0.1 0.001
0.1 0.005
0.1 0.01
0.1 0.05
0.1 0.1
0.1 0.5
0.1 1
0.1 5
0.1 10
0.1 20
0.1 50
0.1 100
0.1473684210526316 1e-05
0.1473684210526316 5e-05
0.1473684210526316 0.0001
0.1473684210526316 0.0005
0.1473684210526316 0.001
0.1473684210526316 0.005
0.1473684210526316 0.01
0.1473684210526316 0.05
0.1473684210526316 0.1
0.1473684210526316 0.5
0.1473684210526316 1
0.1473684210526316 5
0.1473684210526316 10
0.1473684210526316 20
0.1473684210526316 50
0.1473684210526316 100
0.19473684210526315 1e-05
0.19473684210526315 5e-05
0.19473684210526315 0.0001
0.19473684210526315 0.0005
0.19473684210526315 0.001
0.19473684210526315 0.005
0.19473684210526315 0.01
0.19473684210526315 0.05
0.19473684210526315 0.1
0.19473684210526315 0.5
0.19473684210526315 1
0.19473684210526315 5
0.19473684210526315 10
0.19473684210526315 20
0.19473684210526315 50
0.19473684210526315 100
0.24210526315789474 1e-05
0.24210526315789474 5e-05
0.24210526315789474 0.0001
0.242105263157

In [37]:
# Get best parameters
#with open('xgb_params.txt') as f:
#    lines = f.read()
#xgb_params = ast.literal_eval(lines)
#best_xgb_params = min(xgb_params, key=itemgetter(7))[0:7]
best_xgb_params = (0.1, 1767, 7, 0.8, 0.7, 0.6, 1.0)

with open('metadata/rf_params.txt') as f:
    lines = f.read()
rf_params = ast.literal_eval(lines)
best_rf_params = min(rf_params, key=itemgetter(3))[0:3]

with open('metadata/el_net_params.txt') as f:
    lines = f.read()
el_net_params = ast.literal_eval(lines)
best_el_net_params = min(el_net_params, key=itemgetter(2))[0:2]


In [ ]:
# Retrain 
print('Training XGB')
xgb_model = xgb.XGBRegressor(learning_rate=best_xgb_params[0], 
                             n_estimators=best_xgb_params[1],
                             max_depth=best_xgb_params[2],
                             subsample=best_xgb_params[3],
                             colsample_bytree=best_xgb_params[4],
                             colsample_bylevel=best_xgb_params[5],
                             reg_lambda = best_xgb_params[6],
                             objective='reg:squarederror')

fit_params = {
            'eval_metric': 'rmse',
            'early_stopping_rounds': 10,    
            'eval_set': [(X_val, y_val)]
            }

xgb_model.fit(X_train, y_train, eval_metric=fit_params['eval_metric'],
              early_stopping_rounds=fit_params['early_stopping_rounds'], 
              eval_set=fit_params['eval_set'], 
              verbose=False)

# save model to file
pickle.dump(xgb_model, open("metadata/xgb_model.pickle.dat", "wb"))

print('Training RF')
rf = RandomForestRegressor(n_estimators=best_rf_params[0],
                            max_depth=best_rf_params[1],
                            max_features=best_rf_params[2])
rf.fit(X_train, y_train)

# save model to file
pickle.dump(rf, open("metadata/rf_model.pickle.dat", "wb"))

print('Training Elastic Net')
elastic = ElasticNet(alpha=best_el_net_params[0], l1_ratio=best_el_net_params[1])
elastic.fit(X_train_pca, y_train)

# save model to file
pickle.dump(el_net_model, open("metadata/el_net_model.pickle.dat", "wb"))

Training XGB


/home/stas/anaconda3/envs/comp1/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/home/stas/anaconda3/envs/comp1/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



Training RF


In [ ]:
# Perform search for stacking coefficients
params = [
    (0.3, 0.3, 0.3), (0.4, 0.4, 0.2), (0.5, 0.5, 0), (0.6, 0.4, 0), (0.7, 0.3, 0),
    (0.8, 0.2, 0), (0.9, 0.1, 0), (0.6, 0.3, 0.1), (0.6, 0.2, 0.2), (0.5, 0.4, 0.1)
    (0.5, 0.3, 0.2), (0.4, 0.6, 0), (0.4, 0.5, 0.1), (0.4, 0.4, 0.2), (0.3, 0.7, 0),
    (0.3, 0.6, 0.1), (0.3, 0.5, 0.2)
]

models = {
    'xgb' : pickle.load(open("xgb_model.pickle.dat", "rb")),
    'rf' : pickle.load(open("rf_model.pickle.dat", "rb")),
    'el_net' : pickle.load(open("el_net_model.pickle.dat", "rb"))
}

best_stack_params = parameter_search(X_val, y_val, method='stack', params=params, steps=None, models=models)[
                (0.3, 0.3, 0.3), (0.4, 0.4, 0.2), (0.5, 0.5, 0), (0.6, 0.4, 0), (0.7, 0.3, 0),
                (0.8, 0.2, 0), (0.9, 0.1, 0), (0.6, 0.3, 0.1), (0.6, 0.2, 0.2), (0.5, 0.4, 0.1)
                (0.5, 0.3, 0.2), (0.4, 0.6, 0), (0.4, 0.5, 0.1), (0.4, 0.4, 0.2), (0.3, 0.7, 0),
                (0.3, 0.6, 0.1), (0.3, 0.5, 0.2)
]

In [ ]:
# Retrain on the train + val
X = np.concatenate((X_train, X_val), axis=0)
y = np.concatenate((y_train, y_val), axis=0)

xgb_mod = pickle.load(open("xgb_model.pickle.dat", "rb"))
best_number_trees = xgb_mod.best_ntree_limit
rf_mod =  pickle.load(open("rf_model.pickle.dat", "rb")),
el_net_mod = pickle.load(open("el_net_model.pickle.dat", "rb"))

print('Training XGB')
xgb_model = xgb.XGBRegressor(learning_rate=best_xgb_params[0], 
                             n_estimators=best_number_trees,
                             max_depth=best_xgb_params[2],
                             subsample=best_xgb_params[3],
                             colsample_bytree=best_xgb_params[4],
                             colsample_bylevel=best_xgb_params[5],
                             reg_lambda = best_xgb_params[6],
                             objective='reg:squarederror')

fit_params = {'eval_metric': 'rmse'}

xgb_model.fit(X, y, eval_metric=fit_params['eval_metric'],
              verbose=False)

# save model to file
pickle.dump(xgb_model, open("xgb_model.pickle.dat", "wb"))

print('Training RF')
rf = RandomForestRegressor(n_estimators=best_rf_params[0],
                            max_depth=best_rf_params[1],
                            max_features=best_rf_params[2])
rf.fit(X, y)

# save model to file
pickle.dump(rf, open("rf_model.pickle.dat", "wb"))

print('Training Elastic Net')
elastic = ElasticNet(alpha=best_el_net_params[0], l1_ratio=best_el_net_params[1])
                elastic.fit(X_train_pca, y_train)
    
elastic.fit(X, y)

# save model to file
pickle.dump(el_net_model, open("el_net_model.pickle.dat", "wb"))

In [ ]:
with open('stack_params.txt') as f:
    lines = f.read()
stack_params = ast.literal_eval(lines)
best_stack_params = min(stack_params, key=itemgetter(3))[0:3]

models = {
    'xgb' : pickle.load(open("xgb_model.pickle.dat", "rb")),
    'rf' : pickle.load(open("rf_model.pickle.dat", "rb")),
    'el_net' : pickle.load(open("el_net_model.pickle.dat", "rb")) 
}

coefs = {
    'xgb_coef' : best_stack_params[0],
    'rf_coef' : best_stackmodelsams[1],
    'el_coef' : best_stack_params[2]
}

# Test
print('RMSE is :' + str(np.mean(np.sqrt(mean_squared_error(y_cv, predict_stacked(X_test, models=models, coefs=coefs))))))
print('Adam metric is :' + stadam_metricdam_metric(y_cv, predict_stacked(X_test, models=models, coefs=coefs)))
